In [1]:
import os
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
from dotenv import load_dotenv
from src.utils import *
from src.utils_azure import *


load_dotenv('/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/official/.env')



endpoint = os.environ['AZURE_SHIBA_ENDPOINT_KEY']
subscription_key = os.environ['AZURE_SHIBA_SUBSCRIPTION_KEY']
COMPUTERVISION_CLIENT = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
print("Azure Computer Vision Client is ready to use!")

Azure Computer Vision Client is ready to use!


/home/bu0ai/anaconda3/envs/shiba/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [2]:
image_path = '/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/data_label/sample-03.png'
det_draw_path = r'/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/demo/weight/draw_231229/best.pt'
det_code_path = r'/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/demo/weight/code_231207/best.pt'
model_restore_resolution = '/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/GFPGAN/experiments/pretrained_models/GFPGANv1.3.pth'


image_cad = Image.open(image_path)
COMPUTERVISION_CLIENT = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
bg_upsampler = RealESRGANer(
                            scale=2,
                            model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
                            model=model,
                            tile=400,
                            tile_pad=10,
                            pre_pad=0,
                            half=True) 

# Setup Parammeter for GFPGANer
restorer = GFPGANer(model_path=model_restore_resolution,
                    upscale=2,
                    arch='clean',
                    channel_multiplier=2,
                    bg_upsampler=bg_upsampler)

DET_DRAW_MODEL = load_yolo(det_draw_path)
DET_CODE_MODEL = load_yolo(det_code_path)
COMPUTERVISION_CLIENT = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

/home/bu0ai/anaconda3/envs/shiba/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bu0ai/anaconda3/envs/shiba/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
img_high = upscale_image(image_path,
                         restorer,
                         des_path= '/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/tmpFile/temp_azure/temp_azure_cad')
img_high = Image.open(img_high)

words, boxes = ocr_azure_cad(img_high.copy(), COMPUTERVISION_CLIENT)

Done


In [4]:
def get_basic_info(image_pil: Image, computervision_client: ComputerVisionClient) -> Dict:
    """Get basic information of document

    Args:
        image_pil (Image): pillow image of basic information

    Returns:
        Dict: The basic information 
    """
    words, boxes = ocr_azure_cad(image_pil.copy(), computervision_client)
    data_df = pd.DataFrame({'words': words, 'boxes': boxes})
    data_df[['left', 'top', 'right', 'bottom']] = data_df['boxes'].apply(pd.Series)
    data_df['words'] = data_df['words'].apply(lambda e: e.replace(' ', ''))

    try:
        construct_type = data_df[data_df.words.str.contains('ids', case=False)]
        construct_type_str = construct_type.words.tolist()[0]
    except:
        construct_type_str = ''

    try:
        # 分 譲 住 宅 建 築 販 売 設 計 施 工
        title_builder_name = process.extractOne("分 譲 住 宅 建 築 販 売 設 計 施 工", words, scorer=fuzz.WRatio)
        title_builder_name_idx = title_builder_name[2]
        title_builder_name_box = boxes[title_builder_name_idx]
        title_builder_name_width = (title_builder_name_box[2] - title_builder_name_box[0]) * 0.5
        
        match_builder_name = data_df[(data_df.left < (title_builder_name_box[2] + title_builder_name_width)) & (data_df.right > title_builder_name_box[0]) &
                                    (data_df.top > title_builder_name_box[1])]
        match_builder_name.sort_values(by='top', inplace=True)
        match_builder_name = match_builder_name[(match_builder_name.iloc[0].top < match_builder_name.bottom) & (match_builder_name.iloc[0].bottom > match_builder_name.top)]
        match_builder_name_str = ' '.join(match_builder_name.sort_values(by='left').words.tolist())
    except:
        match_builder_name_str = ''

    try:
        anken_title = process.extractOne("TITLE", words, scorer=fuzz.WRatio)
        title_anken_idx = anken_title[2]
        title_anken_box = boxes[title_anken_idx]
        title_anken_width = (title_anken_box[2] - title_anken_box[0]) * 4
        match_anken = data_df[(data_df.left < (title_anken_box[2] + title_anken_width)) & 
                            (data_df.right > title_anken_box[0]) &
                            (data_df.top > title_anken_box[1])]
        match_anken.sort_values(by='top', inplace=True)
        match_anken = match_anken[(match_anken.iloc[0].top < match_anken.bottom) & (match_anken.iloc[0].bottom > match_anken.top)]
        match_anken_str = ' '.join(match_anken.sort_values(by='left').words.tolist())
    except:
        match_anken_str = ''

    return {"construct_type": [construct_type_str],
            "builder_name": [match_builder_name_str],
            "anken": [match_anken_str],
            "ocr": {"words": words, "boxes": boxes}}

In [5]:
result_draw_df = yolo_inference(img_high, DET_DRAW_MODEL, type='draw')
raw_code_df_full = pd.DataFrame(columns=['boxes','cls','angle'])
des_path = '/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/tmpFile/temp_azure/temp_azure_cad_restore_2/'

try:
    os.remove(des_path)
    os.makedirs(f"{des_path}", exist_ok=True)
except:
    os.makedirs(f"{des_path}", exist_ok=True)

count = 1

for temp in result_draw_df.boxes:
    x1, y1, x2, y2 = temp
    image_specified_cad = img_high.crop((x1,y1,x2,y2))
    name_specified = f'image_specified_{x1}_{y1}_{x2}_{y2}.jpg'
    path_specified = f'/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/tmpFile/temp_azure/temp_azure_cad_restore/{name_specified}'
    path_specified_restore = f'/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/tmpFile/temp_azure/temp_azure_cad_restore_2/{name_specified}'
    image_specified_cad.save(path_specified)
    
    image_specified_restore = upscale_image(image_path= path_specified,
                                            restorer= restorer,
                                            is_saved= True,
                                            des_path= des_path)
    
    image_specified_restore = Image.open(path_specified_restore)
    raw_code_df = yolo_inference(image_specified_restore, DET_CODE_MODEL, type= 'code')
    # raw_code_df_full = pd.concat([raw_code_df, raw_code_df], ignore_index=True)
    floor = count
    # code_df = raw_code_df[raw_code_df.boxes.apply(lambda b: calc_ovl(b, ))]

base_image_pil, draw_location_list = split_images(img_high.copy(), result_draw_df)

base_info = get_basic_info(base_image_pil, COMPUTERVISION_CLIENT)

Done
Done


/tmp/ipykernel_2899090/423759645.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_builder_name.sort_values(by='top', inplace=True)
/tmp/ipykernel_2899090/423759645.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_anken.sort_values(by='top', inplace=True)


In [6]:
basic_dict, floor_dict = {}, {}
raw_code_df = yolo_inference(img_high, DET_CODE_MODEL, type='code')
base_image_pil, draw_location_list = split_images(img_high.copy(), result_draw_df)
basic_info_dict = get_basic_info(base_image_pil, COMPUTERVISION_CLIENT)
basic_info_dict

/tmp/ipykernel_2899090/423759645.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_builder_name.sort_values(by='top', inplace=True)
/tmp/ipykernel_2899090/423759645.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_anken.sort_values(by='top', inplace=True)


{'construct_type': ['IDS-Ⅵ型'],
 'builder_name': ['タクトホーム株式会社'],
 'anken': ['3941千葉市大膳野町9棟'],
 'ocr': {'words': ['横架材接合金物 特記',
   'ZATI またはZATⅢ取付不可の',
   '耐力壁及び準耐力壁',
   '注端部の接合方法',
   '接合部種類',
   '短期基準耐力が金物(片面1枚)+プレカット継手で10.4kV',
   '進手1-1',
   'クギ止め短冊金物',
   '柱頭,柱脚に関してはハイパースリムⅡ で対応',
   'A1',
   '壁線位置',
   '短期基準耐力が6.2KN以上の金物',
   'ハイパースリムⅡ:1本',
   '柱接合 2',
   '短期基準耐力が金物(並列2枚又は両面)+プレカット継手で17.8kN',
   '---------',
   '壁線外壁量位置',
   '---------------------------------------------',
   '(1階 柱頭,2階 柱脚のみ)',
   '継手1-2',
   'A2',
   '√ x',
   '-------',
   'クギ止め短冊金物',
   '可変間仕切壁',
   '短期基準耐力が6.2KN以上の金物',
   '金 物月 例',
   '短期基準耐力が金物+ブレカット継手で44.8kN以上',
   'ハイバースリムⅡ:1本',
   '柱接合 2',
   '熊手 2',
   'W短冊金物',
   'B',
   '短期基準耐力が10. 8KN以上の金物',
   '短期基準耐力が金物×I+プレカット仕口で10. 1kV以上',
   'ミドルコーナー15合板タイプ',
   '柱拔合3',
   '仕口1-1',
   'レスビー羽子板ボルト、ハイパーウイング',
   'CI',
   '短期基準耐力が15. 0KN以上の金物',
   '柱脚金物',
   '柱接合 4',
   '仕口1-2',
   '短期基準耐力が金物×2+プレカット仕口で15.9kN以上',
   '02',
   'レスビー羽子板ボルト、ハイパーウイング',
   '上記以外で金物が取り付けられる

In [8]:
draw_information_dict = {}
for idx, draw_location in enumerate(draw_location_list):
    floor_str = str(idx)
    
    code_df = raw_code_df[raw_code_df.boxes.apply(lambda b: calc_ovl(b, draw_location) > 0.4)]
    code_df.reset_index(drop=True, inplace=True)
    num_cell, cell_width, cell_height = get_cell_size(code_df)
    num_cell = int(num_cell)
    concat_image_pil = gen_grid_image(img_high, code_df, num_cell, cell_width, cell_height)
    draw_words, draw_boxes = ocr_azure_cad(concat_image_pil.copy(), COMPUTERVISION_CLIENT)
    extract_dict = mapping_code(img_high, draw_words, draw_boxes, code_df,
                                num_cell, cell_width, cell_height)
    
    # extract_dict = refine_format(extract_dict, floor_str)

In [9]:
extract_dict

{'digit_code': [24,
  19,
  8,
  15,
  11,
  35,
  18,
  29,
  25,
  12,
  22,
  1,
  20,
  6,
  23,
  34,
  33,
  21,
  7,
  27,
  10,
  2,
  36,
  16,
  9,
  37,
  14,
  13,
  28,
  3,
  17,
  26,
  31],
 'code': ['A',
  'A',
  '90MA',
  '90MA',
  '90MH',
  'NT',
  'SOL-L',
  'AX',
  'AX',
  '90MK09',
  'A',
  'H',
  '',
  'A',
  'H',
  'A',
  'M(150中心用)',
  '5Z11-R',
  'A',
  'KH11-L',
  '90MA',
  'A',
  'POMNTW',
  '90MA',
  '90MH',
  'NT',
  'DOMH',
  '90MA',
  'H',
  'KH11',
  'A',
  'AL',
  'H']}